<h1>Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto</h1>

<h3>This NOTEBOOK contains Assignment 1 and Assignment 2 from the graded assignment for week 3.</h3>

<h4><b>ASSIGNMENT 1</b> : Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe.</h4>

<h3>Install packages</h3>

In [2]:
!conda install -c conda-forge geopy --yes
!conda install -c conda-forge beautifulsoup4 --yes
!conda install -c conda-forge urllib3 --yes


Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.10

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          90 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.20.0-py_0 conda-forge


geopy-1.20.0         | 57 KB     | ##################################### | 100% 
geographiclib-1.49   | 

In [3]:
!conda install -c conda-forge geocoder --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.10

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    click-7.0                  |             py_0          61 KB  conda-forge
    ratelim-0.1.6              |             py_2           6 KB  conda-forge
    geocoder-1.38.1            |             py_1          53 KB  conda-forge
    future-0.17.1              |        py36_1000         701 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         821 KB

The following NEW packages will be INSTALLED:

    future:   0.17.1-py36_1000 co

<h3>Import libraries</h3>

In [2]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

from bs4 import BeautifulSoup
import urllib3

import geocoder # import geocoder

print('Libraries imported.')

Libraries imported.


<h3>Read and parse html page and create beautiful soup object. Convert the postal table data from BeautifulSoup object to a list called postal_code_table_data</h3>

In [9]:
http = urllib3.PoolManager()
postal_codes_page = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
response = http.request('GET', postal_codes_page)
soup = BeautifulSoup(response.data)

postal_code_table = soup.find('table', attrs={'class':'wikitable sortable'})
postal_code_table_body = postal_code_table.find('tbody')
postal_code_table_data = []
rows = postal_code_table_body.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    postal_code_table_data.append([ele for ele in cols if ele])

print('First 5 rows of the Canada postal code table,')
print(np.array(postal_code_table_data)[0:5])

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


First 5 rows of the Canada postal code table,
[list([]) list(['M1A', 'Not assigned', 'Not assigned'])
 list(['M2A', 'Not assigned', 'Not assigned'])
 list(['M3A', 'North York', 'Parkwoods'])
 list(['M4A', 'North York', 'Victoria Village'])]


<h3>Create a data frame from the postal codes list : postal_code_table_data. Our data frame is called df henceforth.</h3>

In [11]:
df = pd.DataFrame(postal_code_table_data)
df.columns = ['PostalCode','Borough', 'Neighborhood']
df.drop(0,0, inplace=True)
print(df.shape)
df.head(15)

(288, 3)


,PostalCode,Borough,Neighborhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,Not assigned
10,M8A,Not assigned,Not assigned


<h3>Data frame wrangling. Convert data to final expected data frame structure applying rules listed below :</h3>
<ul>
    <li>remove rows with Borough as 'Not assigned'</li>
    <li>replace 'Not assigned' in Neighborhood with value from cell Borough</li>
    <li>combine same postal code rows by concatenating the neighborhoods</li>
</ul>

In [12]:
# 1
df = df[df['Borough'] != 'Not assigned']

# 2
df.loc[df['Neighborhood'] == 'Not assigned', 'Neighborhood'] = df.loc[df['Neighborhood'] == 'Not assigned', 'Borough'] 

# 3
df['Neighborhood'] = df.groupby(['PostalCode','Borough'])['Neighborhood'].transform(lambda x: ','.join(x))
df.drop_duplicates(inplace=True)

#print('Print an entry to view concatenation = ', df[df['PostalCode'] == 'M9V'])
df.reset_index(drop=True, inplace=True)
df.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson,Garden District"


In [13]:
print('Total postal codes after cleaning data = ', df.shape[0])

Total postal codes after cleaning data =  103


In [ ]:
# initialize your variable to None
#postal_code_values = df['PostalCode'].values
#location_coods = np.array([[]])

#for postal_code in postal_code_values:
    # loop until you get the coordinates
    #lat_lng_coords = None
    #counter = 1
    #while(lat_lng_coords is None and counter <= 10):
      #g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
      #lat_lng_coords = g.latlng
      #if(lat_lng_coords is not None):
          #location_coods.append(lat_lng_coords)

#print('Total number of coordinates = ', location_coods.size)            
#latitude = lat_lng_coords[0]
#longitude = lat_lng_coords[1]

<h4><b>ASSIGNMENT 2</b> : Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood. I tried the above commented code however could not retrieve the coordinates from geocoder hence using the CSV below :</h4>

In [14]:
df_postal_code_lat_lng = pd.read_csv('Geospatial_Coordinates.csv')

In [15]:
print('Read postal codes from coordinates file = ',df_postal_code_lat_lng.shape[0])
df_postal_code_lat_lng.columns = ['PostalCode', 'Latitude', 'Longitude']
df = df.join(df_postal_code_lat_lng.set_index('PostalCode'), on='PostalCode')
df.head()

Read postal codes from coordinates file =  103


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


In [16]:
df.to_csv('Toronto_Neighborhood_Lat_Lng.csv', index=False)